In [1]:
import sys
import pandas as pd
import numpy as np
import sklearn
import requests
import re
import time

In [2]:
#format URL, adding appropriate headers
def URL_format(url):
    http_flag = 0
    www_flag = 0
    new_url = ""
    
    if "http" not in url[:4]:
        http_flag = 1
    if "www" not in url[:11]:
        www_flag = 1

    if http_flag == 1:
        try:
            r = requests.get('https://'+ url, timeout=10)
            new_url = "https://"
        except:
            new_url = "http://"
            
        if www_flag == 1:
            new_url = new_url + "www."
            
        new_url = new_url+url
    else:
        if www_flag == 1:
            if url[:5] == "https":
                new_url = url[:8] + "www." + url[8:]
            elif url[:4] == "http":
                new_url = url[:7] + "www." + url[7:]
        else:
            new_url = url
    time.sleep(1/4)
    return new_url

#Length of URL
def URL_len(url):
    return len(url)

#Length of Hostname
def hostname_len(url):
    #assumes we start with ://www....
    matches = re.findall("://www.([\w\-\.]+)", str(url))
    result = 0
    if (len(matches) >= 1):
        result = len(matches[0])
    return result

#Length of path
def path_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url))[0])
    return match_len

#Length of first directory
def first_dir_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url))[0])
    return match_len
    
#Length of top level domain
def top_level_len(url):
    domains = re.findall("://www.([\w\-\.]+)", str(url))
    if len(domains) != 0:
        tld = domains[0].split('.')
        return len(tld[-1])
    return 0

#Count of '-'
def count_dash(url):
    dashes = 0
    for char in url:
        if char == '-':
            dashes += 1     
    return dashes

#Count of '@'
def count_at(url):
    at = 0
    for char in url:
        if char == '@':
            at += 1     
    return at

#Count of '?'
def count_q(url):
    q = 0
    for char in url:
        if char == '?':
            q += 1     
    return q

#Count of '%'
def count_percent(url):
    percent = 0
    for char in url:
        if char == '%':
            percent += 1     
    return percent

#Count of '.'
def count_dot(url):
    dot = 0
    for char in url:
        if char == '.':
            dot += 1     
    return dot

#Count of '='
def count_equal(url):
    eq = 0
    for char in url:
        if char == '=':
            eq += 1     
    return eq

#Count of ';'
def count_colon(url):
    colon = 0
    for char in url:
        if char == ';':
            colon += 1     
    return colon

#Count of 'www'
def count_www(url):
    www = 0
    www = len(re.findall("www", str(url)))
    return www

#Count of numbers
def count_numbers(url):
    numbers = sum(c.isdigit() for c in url)
    return numbers

#Count of letters
def count_letters(url):
    letters = sum(c.isalpha() for c in url)
    return letters

#Count of directories
def dir_count(url):
    dirs = 0
    for char in url:
        if char == '/':
            dirs = dirs + 1
    #assumes that it starts with ://www....
    return dirs - 2

#Count of single letter directories
def single_letter_dir(url):
    sldir = 0
    sldir = len(re.findall("\/[\w]\/", str(url)))
    return sldir
                
#Count of queries
def query_count(url):
    count = 0
    count = len(re.findall("\?\w+(&?\w+)*", str(url)))
    return count

#Ratio of uppercase to lowercase letters
def ratio_upper_lower(url):
    upperSum = 0
    lowerSum = 0
    for char in url:
        if char.isupper():
            upperSum = upperSum + 1
        if char.islower():
            lowerSum = lowerSum + 1
    # if url has 0 of one, this will return 0
    if lowerSum == 0:
        return upperSum
    return upperSum / lowerSum

#IP vs not
def is_ip(url):
    if len(re.findall("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", str(url))) > 0:
        return 1
    else:
        return 0
    
# Shortened or not
def is_shortened(url):
    shortened = { "bit.ly", "tinyurl", "goo.gl", "ow.ly", "t.co", 
                 "tiny.cc", "bit.do", "shorte.st", "cutt.ly", "clkim"}
    
    if any(shrt in url for shrt in shortened):
        return 1
    else:
        return 0

#HTTP vs. HTTPS
#Returns 1 if HTTPS, 0 if HTTP
def is_https(url):
    if url[:5] == 'https':
        return 1
    elif url[:4] == 'http':
        return 0


In [3]:
#Read csv data using pandas
majestic_million=pd.read_csv('majestic_million.csv')
phishtank=pd.read_csv('phistank_verified_online_data.csv')

#Only take the first 10,000 values of the majestic million values
majestic_million=majestic_million.head(10000)

#Rows are store in res[] after being processed
res=[]

malicious=0
benign=1


In [4]:
for index,row in majestic_million.iterrows():
    #print(row.IDN_Domain)
    url = URL_format(row.IDN_Domain)
    print(index)
    print(url)

    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,benign,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])
  
result_column_names = ['url','malicious','url_res','hostname_res','path_res','first_dir_res','top_level_res','dash_res','at_res','q_res','percent_res','dot_res','equal_res','colon_res','www_res','numbers_res','letters_res','dir_res','single_letter_res','query_res','ratio_upper_lower_res','ip_res','shortened_res','http_res']

result_majestic = pd.DataFrame(res,columns=result_column_names)
result_majestic.to_csv('majestic_out.csv', sep=',',index=False)

0
https://www.google.com
1
https://www.facebook.com
2
https://www.youtube.com
3
https://www.twitter.com
4
https://www.instagram.com
5
https://www.linkedin.com
6
https://www.microsoft.com
7
https://www.apple.com
8
https://www.wikipedia.org
9
https://www.googletagmanager.com
10
https://www.youtu.be
11
https://www.en.wikipedia.org
12
https://www.plus.google.com
13
https://www.pinterest.com
14
https://www.play.google.com
15
https://www.vimeo.com
16
https://www.maps.google.com
17
https://www.adobe.com
18
https://www.goo.gl
19
https://www.wordpress.com
20
https://www.bit.ly
21
https://www.itunes.apple.com
22
https://www.github.com
23
https://www.wordpress.org
24
https://www.blogspot.com
25
https://www.docs.google.com
26
https://www.player.vimeo.com
27
https://www.amazon.com
28
https://www.mozilla.org
29
https://www.yahoo.com
30
https://www.tumblr.com
31
https://www.drive.google.com
32
https://www.support.google.com
33
https://www.apps.apple.com
34
https://www.flickr.com
35
https://www.europa

290
https://www.businesswire.com
291
https://www.yale.edu
292
https://www.aws.amazon.com
293
https://www.sciencedaily.com
294
https://www.hubspot.com
295
https://www.addons.mozilla.org
296
https://www.vox.com
297
https://www.netflix.com
298
https://www.psu.edu
299
https://www.fb.me
300
https://www.plesk.com
301
https://www.cisco.com
302
https://www.redhat.com
303
https://www.tripod.com
304
https://www.iso.org
305
https://www.tiktok.com
306
https://www.podcasts.apple.com
307
https://www.zdnet.com
308
https://www.books.google.com
309
https://www.gofundme.com
310
https://www.noaa.gov
311
https://www.disqus.com
312
https://www.wikihow.com
313
https://www.cbc.ca
314
https://www.state.gov
315
https://www.upenn.edu
316
https://www.intel.com
317
https://www.feeds.feedburner.com
318
https://www.vkontakte.ru
319
https://www.geocities.com
320
https://www.worldbank.org
321
http://www.cambridge.org
322
https://www.docs.microsoft.com
323
https://www.abc.net.au
324
https://www.guardian.co.uk
325
http

578
http://www.dot.gov
579
https://www.sec.gov
580
https://www.moodle.org
581
https://www.mystrikingly.com
582
https://www.shutterstock.com
583
https://www.gov.cn
584
http://www.sakura.ne.jp
585
https://www.mixcloud.com
586
https://www.matterport.com
587
https://www.scmp.com
588
https://www.apachefriends.org
589
https://www.login.microsoftonline.com
590
https://www.unicef.org
591
https://www.pan.baidu.com
592
https://www.ok.ru
593
https://www.openssl.org
594
https://www.news.com.au
595
https://www.jquery.com
596
https://www.drupal.org
597
https://www.photos.app.goo.gl
598
https://www.wp.me
599
https://www.thehill.com
600
https://www.namecheap.com
601
https://www.is.gd
602
https://www.news.google.com
603
https://www.readthedocs.io
604
https://www.kernel.org
605
https://www.icann.org
606
https://www.nhs.uk
607
https://www.hollywoodreporter.com
608
https://www.ru.wikipedia.org
609
http://www.go.th
610
https://www.weather.com
611
https://www.freebsd.org
612
https://www.getbootstrap.com
613

870
http://www.jiathis.com
871
http://www.nasdaq.com
872
https://www.duckduckgo.com
873
https://www.gamespot.com
874
https://www.tinypic.com
875
https://www.jimdofree.com
876
http://www.seattletimes.com
877
http://www.dedecms.com
878
https://www.medlineplus.gov
879
https://www.garmin.com
880
https://www.patch.com
881
https://www.sony.com
882
https://www.pbase.com
883
https://www.en.wiktionary.org
884
https://www.ibb.co
885
https://www.iheart.com
886
https://www.ucsb.edu
887
https://www.hyatt.com
888
https://www.snopes.com
889
https://www.diigo.com
890
https://www.speedtest.net
891
https://www.nsw.gov.au
892
https://www.lefigaro.fr
893
https://www.amazon.fr
894
https://www.download.cnet.com
895
https://www.zoho.com
896
https://www.ameblo.jp
897
https://www.jetpack.wordpress.com
898
https://www.wikidot.com
899
https://www.ibtimes.com
900
https://www.it.wikipedia.org
901
https://www.craigslist.org
902
https://www.1drv.ms
903
https://www.icloud.com
904
https://www.fifa.com
905
http://www.e

1155
https://www.knowyourmeme.com
1156
https://www.hardened-php.net
1157
https://www.upi.com
1158
https://www.indianexpress.com
1159
http://www.hpe.com
1160
https://www.lse.ac.uk
1161
https://www.storify.com
1162
https://www.generatepress.com
1163
https://www.bluehost.com
1164
https://www.elle.com
1165
https://www.thehindu.com
1166
https://www.buffer.com
1167
http://www.wampserver.com
1168
https://www.uk.linkedin.com
1169
https://www.thinkwithgoogle.com
1170
http://www.baltimoresun.com
1171
https://www.tableau.com
1172
https://www.de.wordpress.org
1173
https://www.visualstudio.com
1174
https://www.channel4.com
1175
https://www.bit.do
1176
https://www.nbc.com
1177
https://www.boingboing.net
1178
https://www.bustle.com
1179
https://www.chinaz.com
1180
https://www.rochester.edu
1181
https://www.axios.com
1182
https://www.axs.com
1183
https://www.wsu.edu
1184
https://www.man7.org
1185
http://www.wixstatic.com
1186
https://www.nationalpost.com
1187
https://www.nzherald.co.nz
1188
https://ww

...

8720
http://www.10010.com
8721
http://www.med.br
8722
http://www.karelia.ru
8723
http://www.sld.cu
8724
http://www.suny.edu
8725
http://www.ustreas.gov
8726
http://www.ad.admitad.com
8727
http://www.elle.fr
8728
http://www.hillaryclinton.com
8729
http://www.christianbook.com
8730
http://www.rust-lang.org
8731
http://www.brenebrown.com
8732
http://www.metronews.ca
8733
http://www.blogstival.com
8734
http://www.alibabagroup.com
8735
http://www.zipcar.com
8736
http://www.digitalspy.co.uk
8737
http://www.palm.com
8738
http://www.allaboutvision.com
8739
http://www.hulkshare.com
8740
http://www.nthu.edu.tw
8741
http://www.medcitynews.com
8742
http://www.enotes.com
8743
http://www.badcreditloans123.com
8744
http://www.northumbria.ac.uk
8745
http://www.dreamworks.com
8746
http://www.lacoste.com
8747
http://www.av-test.org
8748
http://www.cvshome.org
8749
http://www.forumotion.com
8750
http://www.jboss.org
8751
http://www.berkshirehathaway.com
8752
http://www.woorank.com
8753
http://www.freeglo

8994
http://www.newson6.com
8995
http://www.liga.net
8996
http://www.floridahealth.gov
8997
http://www.acsm.org
8998
http://www.syracuse.edu
8999
http://www.foxmovies.com
9000
http://www.cebbank.com
9001
http://www.megatakip.com
9002
http://www.maa.org
9003
http://www.foodmate.net
9004
http://www.techworld.com
9005
http://www.wykop.pl
9006
http://www.hidemyass.com
9007
http://www.riaa.com
9008
http://www.tacobell.com
9009
http://www.podio.com
9010
http://www.installmentpersonalloans.org
9011
http://www.surgeongeneral.gov
9012
http://www.zooniverse.org
9013
http://www.rstyle.me
9014
http://www.amazon.com.mx
9015
http://www.childwelfare.gov
9016
http://www.fox5dc.com
9017
http://www.designsponge.com
9018
http://www.aioblogs.com
9019
http://www.d3js.org
9020
http://www.mufg.jp
9021
http://www.telerama.fr
9022
http://www.uni-giessen.de
9023
http://www.96.lt
9024
http://www.anglican.org
9025
http://www.radisson.com
9026
http://www.bizrate.com
9027
http://www.evolutionwriters.com
9028
http:/

9270
http://www.thefrisky.com
9271
http://www.safekids.org
9272
http://www.hb.afl.rakuten.co.jp
9273
http://www.mensfitness.com
9274
http://www.gnb.ca
9275
http://www.construction.com
9276
http://www.edpillst.com
9277
http://www.beyondblue.org.au
9278
http://www.ivermectin6mg.com
9279
http://www.eventbrite.fr
9280
http://www.canon.jp
9281
http://www.expressandstar.com
9282
http://www.containerstore.com
9283
http://www.oprahdaily.com
9284
http://www.pagesuite-professional.co.uk
9285
http://www.fisch-kopf.com
9286
http://www.fischkopfpartnersuche.de
9287
http://www.fcbayern.com
9288
http://www.focusonthefamily.com
9289
http://www.sfgov.org
9290
http://www.yjbys.com
9291
http://www.cloudera.com
9292
http://www.tech.co
9293
http://www.emerson.edu
9294
http://www.postmates.com
9295
http://www.timeinc.net
9296
http://www.getrevue.co
9297
http://www.michaelmoore.com
9298
http://www.swipnet.se
9299
http://www.sofitel.com
9300
http://www.llvm.org
9301
http://www.tetongravity.com
9302
http://www

9545
http://www.irinnews.org
9546
http://www.landsend.com
9547
http://www.thememazing.com
9548
http://www.onlinefreecourse.net
9549
http://www.competitor.com
9550
http://www.docspal.com
9551
http://www.latex-project.org
9552
http://www.loudersound.com
9553
http://www.simplepie.org
9554
http://www.eurowings.com
9555
http://www.wlu.edu
9556
http://www.nrao.edu
9557
http://www.name.com
9558
http://www.amoblog.com
9559
http://www.datacenterknowledge.com
9560
http://www.nabu.de
9561
http://www.mickartvideo.com
9562
http://www.apta.org
9563
http://www.mapy.cz
9564
http://www.brusselstimes.com
9565
http://www.uniovi.es
9566
http://www.a2hosting.com
9567
http://www.carnival.com
9568
http://www.launchrock.com
9569
http://www.script-stack.com
9570
http://www.pastebin.fun
9571
http://www.calstatela.edu
9572
http://www.jdrf.org
9573
http://www.ratgeberrecht.eu
9574
http://www.hotfrog.com
9575
http://www.diesel.com
9576
http://www.lichess.org
9577
http://www.by.ru
9578
http://www.fincen.gov
9579
ht

9824
http://www.kuow.org
9825
http://www.ossoccer.org
9826
http://www.someecards.com
9827
http://www.tur.br
9828
http://www.snapfish.com
9829
http://www.pf.kakao.com
9830
http://www.sklep.pl
9831
http://www.dhnet.be
9832
http://www.hrblock.com
9833
http://www.hooyoo.com
9834
http://www.1and1.com
9835
http://www.pottermore.com
9836
http://www.csusb.edu
9837
http://www.cebit.de
9838
http://www.nairacircle.com
9839
http://www.domyhomework.pro
9840
http://www.lesscss.org
9841
http://www.theworlds50best.com
9842
http://www.bplans.com
9843
http://www.raconteur.net
9844
http://www.westonaprice.org
9845
http://www.swagbucks.com
9846
http://www.nybg.org
9847
http://www.sciencing.com
9848
http://www.mikado-themes.com
9849
http://www.alison.com
9850
http://www.religioustolerance.org
9851
http://www.weareteachers.com
9852
http://www.walla.co.il
9853
http://www.redbookmag.com
9854
http://www.zynga.com
9855
http://www5a.biglobe.ne.jp
9856
http://www.euskadi.eus
9857
http://www.vlaanderen.be
9858
htt

In [6]:
for index,row in phishtank.iterrows():
    #print("original: " + row.url)
    url = row.url
    #print("formatted: " + url)
    
    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,malicious,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])

result = pd.DataFrame(res,columns=result_column_names)
result.to_csv('out.csv', sep=',',index=False)